# Deploy DeepSeek-R1 with Amazon SageMaker Jumpstart

In this notebook, We will use the ModelBuilder class to deploy DeepSeek-R1 model to SageMaker Endpoints

## Prerequisites. 
You will need access access to a ml.p5e.48xlarge


In [ ]:
#Install Sagemaker python SDK
!pip install --force-reinstall --no-cache-dir sagemaker==2.235.2

In [ ]:
#Import required libraries and initialize session
from sagemaker.serve.builder.model_builder import ModelBuilder 
from sagemaker.serve.builder.schema_builder import SchemaBuilder 
from sagemaker.jumpstart.model import ModelAccessConfig 
from sagemaker.session import Session 
import logging 

sagemaker_session = Session()
 
artifacts_bucket_name = sagemaker_session.default_bucket() 
execution_role_arn = sagemaker_session.get_caller_identity_arn()
 


In [ ]:
js_model_id = "deepseek-llm-r1"

gpu_instance_type = "ml.p5e.48xlarge"

response = "Hello, I'm a language model, and I'm here to help you with your English."

sample_input = {
    "inputs": "Hello, I'm a language model,",
    "parameters": {"max_new_tokens": 128, "top_p": 0.9, "temperature": 0.6},
}

sample_output = [{"generated_text": response}]

schema_builder = SchemaBuilder(sample_input, sample_output)

model_builder = ModelBuilder(
    model=js_model_id,
    schema_builder=schema_builder,
    sagemaker_session=sagemaker_session,
    role_arn=execution_role_arn,
    log_level=logging.ERROR
)

model = model_builder.build()
predictor = model.deploy(
    model_access_configs={js_model_id: ModelAccessConfig(accept_eula=True)},
    accept_eula=True
)

In [ ]:
payload = { "inputs": "I believe the meaning of life is", 
            "parameters": { 
                "max_new_tokens": 64, 
                "top_p": 0.9, 
                "temperature": 0.6 
            } 
        }
predictor.predict(payload)

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()